# 📄 Breakout Processing Pipeline

This stage merges birthdates into the DOM metrics table, filters eligible players, computes breakout year and breakout age, restructures columns, and saves the final dataset.

---

## **1. Merge Birthdates**
- Load the DOM metrics file and birthday master file.
- Merge `birth_date` onto the DOM table using `player_name`.
- Remove duplicate birthday rows where necessary.

---

## **2. Filter Players**
- Convert `draft_year` to numeric.
- Drop all players drafted **before 2004**.

---

## **3. Compute Breakout Year**
Evaluate seasons **Year1 → Year4** and assign the first year meeting any breakout threshold:

- `DOM ≥ 0.20`
- `RDOM ≥ 0.20`
- `PDOM ≥ 0.55`

If no season meets these thresholds, No fallback logic is applied

---

## **4. Compute Breakout Age**
Breakout age is calculated using the player’s birthdate and the breakout year:

\[
\text{breakout\_age} = \frac{\text{Sept 1 of breakout\_year} - \text{birth\_date}}{365.25}
\]

- The reference date is **September 1st of the breakout_year**.
- If `breakout_year` is missing then leave empty

## **5. Column Cleanup & Ordering**
### Columns Removed
- `pick_in_round`
- `pick_overall`
- `pfr_draft_url`

### Final Column Order
1. `pos`
2. `birth_date`
3. `breakout_year`
4. `breakout_age`
5. `Year1` and all subsequent year-by-year fields


In [15]:
import pandas as pd
from pathlib import Path
import numpy as np

# ====== CONFIG ======
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")

PLAYERS_CSV  = ROOT / "data" / "processed" / "players_dom_metrics_filled.csv"
BIRTHDAY_CSV = ROOT / "data" / "processed" / "skill_draftee_birthdays_master.csv"
OUT_CSV      = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout.csv"

print("Players CSV:", PLAYERS_CSV)
print("Birthdays CSV:", BIRTHDAY_CSV)

# ====== LOAD TABLES ======
players   = pd.read_csv(PLAYERS_CSV)
birthdays = pd.read_csv(BIRTHDAY_CSV)

# Expect BIRTHDAY_CSV to have: player_name, birth_date
if "player_name" not in birthdays.columns:
    raise ValueError("BIRTHDAY_CSV must contain a 'player_name' column.")
if "birth_date" not in birthdays.columns:
    raise ValueError("BIRTHDAY_CSV must contain a 'birth_date' column.")

birthdays_small = (
    birthdays[["player_name", "birth_date"]]
    .drop_duplicates(subset=["player_name"])
)

# Merge birth_date into players
if "player_name" not in players.columns:
    raise ValueError("PLAYERS_CSV must contain a 'player_name' column for merge.")

players = players.merge(
    birthdays_small,
    on="player_name",
    how="left",
    validate="m:1"
)

# ====== DROP PLAYERS BEFORE 2004 ======
draft_year_col = None
for c in players.columns:
    norm = c.lower().replace(" ", "_")
    if norm in ("draft_year", "draftyear"):
        draft_year_col = c
        break

if draft_year_col is None:
    raise ValueError("Could not find a draft_year column (draft_year / Draft Year / draftyear).")

players[draft_year_col] = pd.to_numeric(players[draft_year_col], errors="coerce")
before_count = len(players)
players = players[players[draft_year_col] >= 2004].copy()
after_count = len(players)
print(f"Dropped players before 2004: {before_count - after_count} (remaining: {after_count})")

# ====== BREAKOUT YEAR (THRESHOLDS ONLY, NO FALLBACK) ======
# First season where:
#   DOMn  >= 0.20  OR
#   RDOMn >= 0.20  OR
#   PDOMn >= 0.55
# breakout_year = YearN of that season

def compute_breakout_year(row):
    for n in range(1, 5):  # support Year1–Year4 if present
        dom  = row.get(f"DOM{n}")
        rdom = row.get(f"RDOM{n}")
        pdom = row.get(f"PDOM{n}")
        year = row.get(f"Year{n}")

        if pd.isna(year):
            continue

        cond_dom  = pd.notna(dom)  and dom  >= 0.20
        cond_rdom = pd.notna(rdom) and rdom >= 0.20
        cond_pdom = pd.notna(pdom) and pdom >= 0.55

        if cond_dom or cond_rdom or cond_pdom:
            return year

    # No breakout season found under the rules
    return np.nan

players["breakout_year"] = players.apply(compute_breakout_year, axis=1)
players["breakout_year"] = pd.to_numeric(
    players["breakout_year"], errors="coerce"
).astype("Int64")

# ====== BREAKOUT AGE: BIRTHDAY → SEPT 1 OF BREAKOUT YEAR ======
# breakout_age = age (in years) on Sept 1 of breakout_year

players["birth_date_dt"] = pd.to_datetime(players["birth_date"], errors="coerce")

def compute_breakout_age(row):
    birth = row["birth_date_dt"]
    year  = row["breakout_year"]

    if pd.isna(birth) or pd.isna(year):
        return np.nan

    ref_date = pd.Timestamp(int(year), 9, 1)  # Sept 1 of breakout_year
    age_in_years = (ref_date - birth).days / 365.25
    return age_in_years

players["breakout_age"] = players.apply(compute_breakout_age, axis=1)

# Drop helper datetime column
players = players.drop(columns=["birth_date_dt"])

# ====== DROP UNWANTED COLUMNS ======
cols_to_drop = [c for c in ["pick_in_round", "pick_overall", "pfr_draft_url"] if c in players.columns]
if cols_to_drop:
    players = players.drop(columns=cols_to_drop)

# ====== COLUMN REORDERING ======
cols = list(players.columns)

# 1) Identify position column
pos_col = None
for cand in ["pos", "Pos", "position", "Position"]:
    if cand in cols:
        pos_col = cand
        break

# 2) Move birth_date right after position column
if pos_col and "birth_date" in cols:
    cols.remove("birth_date")
    idx = cols.index(pos_col) + 1
    cols.insert(idx, "birth_date")

# 3) Move breakout_year and breakout_age right before Year1
year1_col = "Year1" if "Year1" in cols else None
if year1_col:
    for c in ["breakout_year", "breakout_age"]:
        if c in cols:
            cols.remove(c)
    idx = cols.index(year1_col)
    cols.insert(idx, "breakout_year")
    cols.insert(idx + 1, "breakout_age")

players = players[cols]

# ====== SAVE RESULT ======
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
players.to_csv(OUT_CSV, index=False)

print("Final shape:", players.shape)
print("Saved with birth_date / breakout_year / breakout_age to:", OUT_CSV)

preview_cols = ["player_name"]
if pos_col and pos_col not in preview_cols:
    preview_cols.append(pos_col)
for c in ["birth_date", "breakout_year", "breakout_age", draft_year_col]:
    if c in players.columns and c not in preview_cols:
        preview_cols.append(c)

print(players[preview_cols].head())


Players CSV: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/players_dom_metrics_filled.csv
Birthdays CSV: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/skill_draftee_birthdays_master.csv
Dropped players before 2004: 348 (remaining: 1785)
Final shape: (1785, 224)
Saved with birth_date / breakout_year / breakout_age to: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout.csv
          player_name pos  birth_date  breakout_year  breakout_age  draft_year
348       Eli Manning  QB  1981-01-03           2001     20.659822        2004
349  Larry Fitzgerald  WR  1983-08-31           <NA>           NaN        2004
350     Philip Rivers  QB  1981-12-08           2000     18.732375        2004
351    Kellen Winslow  TE  1983-07-21           <NA>           NaN        2004
352      Roy Williams  WR  1980-08-14           2000     20.049281        2004


## **Add PFF Elusive Rating to Master + Round Decimals**

This cell loads your master player file, reads all  
`rushing_summary_{year}.csv` files from the PFF directory, extracts the  
**elusive_rating**, merges it into the master, rounds all float columns to  
**0.001 precision**, and saves a new enriched CSV.

### **Process Summary**
1. Load master file:  
   `players_dom_birthday_breakout.csv`
2. Load all PFF rushing summary files (2004–2029)
3. Extract:  
   - `player_name`  
   - `elusive_rating`  
   - `year`
4. Merge elusive rating by:  
   - **player_name + breakout_year** (preferred)  
   - fallback: **player_name only**
5. Save output as:  
   `players_dom_birthday_breakout_elusive.csv`


In [17]:
import pandas as pd
from pathlib import Path
import numpy as np

# ========== CONFIG ==========
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")

MASTER_CSV = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout.csv"
PFF_DIR    = ROOT / "data" / "CFB_Data" / "PFF"

OUT_CSV    = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive.csv"

print("Master CSV:", MASTER_CSV)
print("PFF Directory:", PFF_DIR)

# ========== LOAD MASTER FILE ==========
master = pd.read_csv(MASTER_CSV)

if "player_name" not in master.columns:
    raise ValueError("Master CSV must contain a column named 'player_name'.")

# ========== BUILD ELUSIVE TABLE ==========
elusive_frames = []

for year in range(2004, 2030):
    p = PFF_DIR / "Rushing" / f"rushing_summary_{year}.csv"
    if not p.exists():
        continue

    df = pd.read_csv(p)
    df["year"] = year

    # Identify player name column
    name_col = None
    for cand in ["player_name", "Player", "player"]:
        if cand in df.columns:
            name_col = cand
            break
    if name_col is None:
        raise ValueError(f"No player name column found in {p}")

    df = df.rename(columns={name_col: "player_name"})

    # Identify elusive column
    elusive_col = None
    for cand in ["elusive_rating", "Elusive Rating", "Elusive", "elusive"]:
        if cand in df.columns:
            elusive_col = cand
            break
    if elusive_col is None:
        raise ValueError(f"No elusive rating column found in {p}")

    df = df.rename(columns={elusive_col: "elusive_rating"})

    elusive_frames.append(df[["player_name", "elusive_rating", "year"]])

if not elusive_frames:
    raise ValueError("No PFF rushing_summary_{year}.csv files found!")

elusive_table = pd.concat(elusive_frames, ignore_index=True)
elusive_table["year"] = pd.to_numeric(elusive_table["year"], errors="coerce").astype("Int64")

# ========== MERGE ELUSIVE INTO MASTER BY Year1–Year5 ==========
merged = master.copy()

for n in range(1, 6):
    year_col = f"Year{n}"
    out_col  = f"elusive_rating{n}"

    if year_col not in merged.columns:
        continue

    tmp = merged[["player_name", year_col]].copy()
    tmp["year"] = pd.to_numeric(tmp[year_col], errors="coerce").astype("Int64")

    tmp = tmp.merge(
        elusive_table,
        on=["player_name", "year"],
        how="left"
    )

    merged[out_col] = tmp["elusive_rating"]

# ========== REORDER SO elusive_ratingN FOLLOWS YEAR-N METRICS ==========

cols = list(merged.columns)

for n in range(1, 6):
    el_col = f"elusive_rating{n}"
    if el_col not in cols:
        continue

    # remove from current position
    cols.remove(el_col)

    # find *all* metric columns ending with number N
    metric_suffix = str(n)
    metric_cols_n = [c for c in cols if c.endswith(metric_suffix)]

    if not metric_cols_n:
        # fallback: put it right after YearN
        year_pos = cols.index(f"Year{n}") + 1
        cols.insert(year_pos, el_col)
        continue

    # place elusive_ratingN after the LAST metric with suffix N
    last_metric = metric_cols_n[-1]
    insert_pos = cols.index(last_metric) + 1
    cols.insert(insert_pos, el_col)

merged = merged[cols]

# ========== SAVE ==========
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
merged.to_csv(OUT_CSV, index=False)

print("Saved with elusive_rating1–5 placed after corresponding season metrics:", OUT_CSV)

peek_cols = ["player_name"] + [c for c in merged.columns if "elusive" in c]
print(merged[peek_cols].head())


Master CSV: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout.csv
PFF Directory: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/CFB_Data/PFF
Saved with elusive_rating1–5 placed after corresponding season metrics: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive.csv
        player_name  elusive_rating1  elusive_rating2  elusive_rating3  \
0       Eli Manning              NaN              NaN              NaN   
1  Larry Fitzgerald              NaN              NaN              NaN   
2     Philip Rivers              NaN              NaN              NaN   
3    Kellen Winslow              NaN              NaN              NaN   
4      Roy Williams              NaN              NaN              NaN   

   elusive_rating4  elusive_rating5  
0              NaN              NaN  
1              NaN              NaN  
2              NaN         

## **Best + Final Season Averaging**  
Determine each player's *best* and *final qualifying* season based on DOM/RDOM/PDOM, then average all season-based metrics (suffixes 1–5). Outputs averaged metrics using original base names.

### **Logic**
- **best_slot** = season (1–5) with highest **DOMn** value  
- **final_slot** = last season where **DOMn ≥ 0.15**  
- If no season meets threshold → `final_slot = best_slot`  
- Average all metrics with numeric suffixes for `best_slot` and `final_slot`  
- Skip averaging: `DOM+`, `RDOM+`, `PDOM+`  
- Output columns have **no suffix**, e.g. `YPA`, `Y/RR`, `EPA/P`, etc.

In [18]:
import pandas as pd
from pathlib import Path
import numpy as np
import re

# ========== CONFIG ==========
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")

IN_CSV  = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive.csv"
OUT_CSV = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive_best_final_avg.csv"

print("Input CSV :", IN_CSV)
print("Output CSV:", OUT_CSV)

# ========== LOAD ==========
df = pd.read_csv(IN_CSV)

# Identify position column
pos_col = None
for cand in ["pos", "Pos", "position", "Position"]:
    if cand in df.columns:
        pos_col = cand
        break

if pos_col is None:
    raise ValueError("Could not find a position column (pos / Pos / position / Position).")

# ========== HELPER: DETERMINE BEST & FINAL SLOTS ==========
def choose_slots(row):
    """
    For a single player row, determine:
      - best_slot  : season (1..5) with the highest relevant DOM metric
      - final_slot : last season (1..5) meeting the threshold.
                     If none meet the threshold, falls back to best_slot.
    WR/TE -> DOMn, threshold 0.15
    RB    -> RDOMn, threshold 0.15
    QB    -> PDOMn, threshold 0.40
    """
    pos = str(row[pos_col]).upper()

    if pos in ("WR", "TE"):
        metric_prefix = "DOM"
        threshold = 0.15
    elif pos == "RB":
        metric_prefix = "RDOM"
        threshold = 0.15
    elif pos == "QB":
        metric_prefix = "PDOM"
        threshold = 0.40
    else:
        # Fallback: treat as WR/TE style DOM
        metric_prefix = "DOM"
        threshold = 0.15

    values = []
    for n in range(1, 6):
        col = f"{metric_prefix}{n}"
        val = row.get(col, np.nan)
        values.append(val)

    # Determine best_slot (max metric)
    best_slot = np.nan
    best_val = -np.inf
    for n, val in enumerate(values, start=1):
        if pd.notna(val) and val > best_val:
            best_val = val
            best_slot = n

    if best_val == -np.inf:
        # no data at all
        return pd.Series({"best_slot": np.nan, "final_slot": np.nan})

    # Determine final_slot (last slot meeting threshold)
    final_slot = np.nan
    for n, val in enumerate(values, start=1):
        if pd.notna(val):
            if (pos in ("WR", "TE") and val >= threshold) or \
               (pos == "RB" and val >= threshold) or \
               (pos == "QB" and val >= threshold) or \
               (pos not in ("WR","TE","RB","QB") and val >= threshold):
                final_slot = n

    # If no season hits the threshold, fall back to best_slot
    if pd.isna(final_slot):
        final_slot = best_slot

    return pd.Series({"best_slot": best_slot, "final_slot": final_slot})

df[["best_slot", "final_slot"]] = df.apply(choose_slots, axis=1)

# ========== IDENTIFY METRIC BASE NAMES (WITH NUMERIC SUFFIXES) ==========
# We care about any column that ends with 1..5, like "DOM1", "YPA3", "Break %2"
metric_cols = [c for c in df.columns if re.search(r"\d$", c)]

base_to_suffixes = {}
for c in metric_cols:
    suffix = c[-1]
    if suffix not in "12345":
        continue
    base = c[:-1]  # strip the trailing digit
    base_to_suffixes.setdefault(base, set()).add(int(suffix))

# We must NOT average DOM+, RDOM+, PDOM+
skip_bases = {"DOM+", "RDOM+", "PDOM+"}
bases = sorted(b for b in base_to_suffixes.keys() if b not in skip_bases)

# ========== BUILD AGGREGATED (BEST+FINAL AVERAGE) DF ==========
# Keep non-suffixed columns (identifiers etc.), drop helper slots later.
id_cols = [c for c in df.columns if not re.search(r"\d$", c)]
id_cols = [c for c in id_cols if c not in ["best_slot", "final_slot"]]

agg = df[id_cols].copy()

def avg_for_base(base: str) -> pd.Series:
    """
    For each row, average base{best_slot} and base{final_slot},
    ignoring NaNs. If both NaN, result is NaN.
    """
    out_vals = []
    for idx in df.index:
        bslot = df.at[idx, "best_slot"]
        fslot = df.at[idx, "final_slot"]

        vals = []

        if pd.notna(bslot):
            col_b = f"{base}{int(bslot)}"
            if col_b in df.columns:
                v = df.at[idx, col_b]
                if pd.notna(v):
                    vals.append(v)

        if pd.notna(fslot):
            col_f = f"{base}{int(fslot)}"
            if col_f in df.columns:
                v = df.at[idx, col_f]
                if pd.notna(v):
                    vals.append(v)

        if vals:
            out_vals.append(float(np.mean(vals)))
        else:
            out_vals.append(np.nan)

    return pd.Series(out_vals, index=df.index)

for base in bases:
    agg[base] = avg_for_base(base)

# ========== SAVE RESULT ==========
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
agg.to_csv(OUT_CSV, index=False)

print("Saved averaged best+final season metrics to:", OUT_CSV)
print("Columns in output (truncated):")
print(list(agg.columns)[:40])


Input CSV : /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive.csv
Output CSV: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg.csv
Saved averaged best+final season metrics to: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg.csv
Columns in output (truncated):
['player_name', 'pos', 'birth_date', 'team', 'conf', 'draft_year', 'draft_round', 'breakout_year', 'breakout_age', '10+', 'ADJ%', 'BTT%', 'Break %', 'CC%', 'CTPRR', 'Comp%', 'DAA', 'DOM', 'DRP%', 'Drop%', 'EPA/P', 'FUM', 'MTF', 'MTF/A', 'PDOM', 'RDOM', 'REC%', 'TWP%', 'UCTPRR', 'Y/REC', 'Y/RR', 'YAC/R', 'YCO/A', 'YPA', 'YPC', 'YPR', 'Year', 'aDOT', 'aYPTPA', 'elusive_rating']


## **Conference-Adjusted DOM Metrics**  
Apply conference strength multipliers to DOM/RDOM/PDOM, generate DOM+/RDOM+/PDOM+, round all float metrics, and drop unused raw team/player stat columns.

### **Logic**
- Map each row's `conf` to a **conference multiplier** (`conf_mult`) using your custom `CONF_MULT` dictionary  
- Compute:
  - `DOM+  = DOM  * conf_mult`  
  - `RDOM+ = RDOM * conf_mult`  
  - `PDOM+ = PDOM * conf_mult`  
- Round **all float columns** to **0.001 precision**  
- Reorder columns so:
  - `DOM` is immediately followed by `DOM+`  
  - `RDOM` is immediately followed by `RDOM+`  
  - `PDOM` is immediately followed by `PDOM+`  
- Drop unused raw stat columns and helpers:  
  `pass_tds, pass_yds, rec_tds, rec_yds, rush_tds, rush_yds, team_pr_tds, team_pr_yds, team_ru_tds, team_ru_yds, conf_mult, Year`  
- Save cleaned, conference-adjusted metrics to  
  `players_dom_birthday_breakout_elusive_best_final_avg_conf.csv`


In [20]:
import pandas as pd
from pathlib import Path
import numpy as np

# ========== CONFIG ==========
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")

IN_CSV  = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive_best_final_avg.csv"
OUT_CSV = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive_best_final_avg_conf.csv"

print("Input CSV :", IN_CSV)
print("Output CSV:", OUT_CSV)

# ========== LOAD ==========
df = pd.read_csv(IN_CSV)

# ========== CONFERENCE MULTIPLIER MAP ==========
CONF_MULT = {
    "SEC": 1.0,
    "BIG TEN": 1.0, "BIG 10": 1.0, "B1G": 1.0,
    "BIG 12": 0.95,
    "ACC": 0.95,
    "PAC-12": 0.95, "PAC 12": 0.95,
    "AAC": 0.85, "AMERICAN": 0.85,
    "MOUNTAIN WEST": 0.76, "MWC": 0.76,
    "SUN BELT": 0.76,
    "MAC": 0.70,
    "C-USA": 0.70, "CUSA": 0.70, "CONFERENCE USA": 0.70,
    "INDEPENDENT": 0.60, "IND": 0.60,
    "OTHER": 0.60, "FCS": 0.60, "NON-FBS": 0.60,
}

def get_conf_multiplier(conf):
    if pd.isna(conf):
        return 0.60
    key = str(conf).strip().upper()
    return CONF_MULT.get(key, 0.60)

# ========== APPLY MULTIPLIERS ==========
if "conf" not in df.columns:
    raise ValueError("Expected a 'conf' column.")

df["conf_mult"] = df["conf"].apply(get_conf_multiplier)

# ---- NEW: ensure we replace old averaged DOM+/RDOM+/PDOM+ ----
for col in ["DOM+", "RDOM+", "PDOM+"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

for base in ["DOM", "RDOM", "PDOM"]:
    if base in df.columns:
        df[f"{base}+"] = df[base] * df["conf_mult"]

# ========== ROUND FLOATS ==========
float_cols = df.select_dtypes(include=["float64", "float32"]).columns
df[float_cols] = df[float_cols].round(3)

print(f"Rounded {len(float_cols)} float columns to 0.001 precision.")

# ========== COLUMN REORDERING: Place DOM+ next to DOM, etc. ==========
cols = list(df.columns)

def move_next(original, plus):
    """Move plus-column immediately after the original column, if both exist."""
    if original in cols and plus in cols:
        cols.remove(plus)
        idx = cols.index(original) + 1
        cols.insert(idx, plus)

move_next("DOM", "DOM+")
move_next("RDOM", "RDOM+")
move_next("PDOM", "PDOM+")

df = df[cols]

# ========== DROP UNUSED TEAM / RAW STAT COLUMNS + conf_mult ==========
drop_cols = [
    "pass_tds", "pass_yds",
    "rec_tds", "rec_yds",
    "rush_tds", "rush_yds",
    "team_pr_tds", "team_pr_yds",
    "team_ru_tds", "team_ru_yds",
    "conf_mult", "Year"
]
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

# ========== SAVE ==========
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_CSV, index=False)

print("Saved metrics (with DOM+/RDOM+/PDOM+ and cleaned columns) to:", OUT_CSV)
if all(c in df.columns for c in ["player_name", "DOM", "DOM+", "RDOM", "RDOM+", "PDOM", "PDOM+"]):
    print(df[["player_name", "DOM", "DOM+", "RDOM", "RDOM+", "PDOM", "PDOM+"]].head())


Input CSV : /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg.csv
Output CSV: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg_conf.csv
Rounded 47 float columns to 0.001 precision.
Saved metrics (with DOM+/RDOM+/PDOM+ and cleaned columns) to: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg_conf.csv
        player_name    DOM   DOM+   RDOM  RDOM+   PDOM  PDOM+
0       Eli Manning  0.000  0.000  0.039  0.039  0.973  0.973
1  Larry Fitzgerald  0.000  0.000  0.000  0.000  0.000  0.000
2     Philip Rivers -0.000 -0.000  0.146  0.139  0.980  0.931
3    Kellen Winslow  0.000  0.000  0.000  0.000  0.000  0.000
4      Roy Williams  0.406  0.284  0.026  0.018  0.005  0.004


## **Speed & BMI Enrichment (40yd Data Merge)**  
Attach best 40-yard dash time and BMI to the master player table.

### **Logic**
- Load:
  - Master player table:  
    `players_dom_birthday_breakout_elusive_best_final_avg_conf.csv`  
  - 40-yard results:  
    `40yd_time_2000_2025.csv`  
- Normalize player name columns in both tables to a common `player_name` field  
- Detect key measurement columns in the 40yd file:
  - Forty time: `forty_s` / `40` / `40yd` / `40_time` / `forty`  
  - Height (inches): `height_in`  
  - Weight (pounds): `weight_lb`  
- Clean and prepare 40yd data:
  - Coerce forty time, height, and weight to numeric  
  - Drop rows with missing forty time  
  - For players with **multiple** recorded times, keep the **fastest** (minimum forty time per `player_name`)  
- Compute derived metrics:
  - `BMI   = (weight_lb / height_in^2) * 703`  
  - `speed = forty_time` (lower = faster)  
- Reduce 40yd table to:  
  `player_name, speed, BMI`  
- Left-join `speed` and `BMI` onto the master table by `player_name`  
- Round **all float columns** to **0.001 precision**  
- Save the enriched master file to:  
  `players_dom_birthday_breakout_elusive_best_final_avg_conf_speed_bmi.csv`


In [23]:
import pandas as pd
from pathlib import Path
import numpy as np

# ========== CONFIG ==========
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")

MASTER_CSV = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive_best_final_avg_conf.csv"
FORTY_CSV  = ROOT / "data" / "scraper" / "40yd_time_2000_2025.csv"

OUT_CSV    = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive_best_final_avg_conf_speed_bmi.csv"

print("Master CSV:", MASTER_CSV)
print("40yd CSV  :", FORTY_CSV)
print("Out CSV   :", OUT_CSV)

# ========== LOAD ==========
master = pd.read_csv(MASTER_CSV)
forty  = pd.read_csv(FORTY_CSV)

# ---------- Normalize player name columns ----------
def find_first_col(cols, candidates):
    cols_lower = {c.lower(): c for c in cols}
    for cand in candidates:
        if cand.lower() in cols_lower:
            return cols_lower[cand.lower()]
    return None

master_name_col = find_first_col(master.columns, ["player_name", "player", "Player"])
forty_name_col  = find_first_col(forty.columns,  ["player_name", "player", "Player"])

if master_name_col is None or forty_name_col is None:
    raise ValueError("Missing player_name column in one of the CSVs.")

master = master.rename(columns={master_name_col: "player_name"})
forty  = forty.rename(columns={forty_name_col: "player_name"})

# ---------- Detect columns ----------
forty_col  = find_first_col(forty.columns, ["forty_s", "40", "40yd", "40_time", "forty"])
height_col = find_first_col(forty.columns, ["height_in"])
weight_col = find_first_col(forty.columns, ["weight_lb"])

if forty_col is None:
    raise ValueError("Could not find forty-time column in 40yd CSV (forty_s).")
if height_col is None:
    raise ValueError("Could not find 'height_in' column in 40yd CSV.")
if weight_col is None:
    raise ValueError("Could not find 'weight_lb' column in 40yd CSV.")

# ---------- Clean forty dataset ----------
forty = forty.copy()
forty[forty_col]  = pd.to_numeric(forty[forty_col], errors="coerce")
forty[weight_col] = pd.to_numeric(forty[weight_col], errors="coerce")
forty[height_col] = pd.to_numeric(forty[height_col], errors="coerce")

# Drop rows missing forty time
forty = forty[forty[forty_col].notna()]

# For players with multiple forty times, use best (fastest)
idx_fastest = forty.groupby("player_name")[forty_col].idxmin()
forty_best = forty.loc[idx_fastest].copy()

# ---------- Compute BMI + speed ----------
# BMI = (weight_lb / height_in^2) * 703
with np.errstate(divide="ignore", invalid="ignore"):
    forty_best["BMI"] = (forty_best[weight_col] / (forty_best[height_col] ** 2)) * 703

forty_best["speed"] = forty_best[forty_col]

forty_keep = forty_best[["player_name", "speed", "BMI"]]

# ---------- MERGE ----------
merged = master.merge(forty_keep, on="player_name", how="left")

# ---------- ROUND ----------
float_cols = merged.select_dtypes(include=["float64", "float32"]).columns
merged[float_cols] = merged[float_cols].round(3)

# ---------- SAVE ----------
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
merged.to_csv(OUT_CSV, index=False)

print("Saved:", OUT_CSV)
print(merged[["player_name", "speed", "BMI"]].head())


Master CSV: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg_conf.csv
40yd CSV  : /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/scraper/40yd_time_2000_2025.csv
Out CSV   : /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg_conf_speed_bmi.csv
Saved: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg_conf_speed_bmi.csv
        player_name  speed     BMI
0       Eli Manning   4.90  26.204
1  Larry Fitzgerald   4.48  28.120
2     Philip Rivers   5.08  27.152
3    Kellen Winslow   4.55  30.549
4      Roy Williams   4.48  27.216


## **Master Feature Canonicalization & SpeedxBMI**  
Standardize feature names using canonical mappings and derive a combined Speed×BMI metric for the master list.

### **Logic**
- Configure project paths:
  - Set `ROOT` to the Dynasty repo root and add it to `sys.path` (if missing) so `src` modules can be imported.
  - Load input master file:  
    `players_dom_birthday_breakout_elusive_best_final_avg_conf_speed_bmi.csv`  
  - Define output file:  
    `master_list.csv`

- Import canonical feature metadata from `src.utils`:
  - `BASE_FEATURES`: list of canonical feature names to enforce  
  - `ALIASES`: mapping from canonical name → known alternate column names  
  - (Also imports `INTERACTIONS`, though not used directly in this cell)

- Normalize column names for fuzzy matching:
  - Define `norm(s)` to:
    - Lowercase
    - Trim spaces
    - Replace `%` → `pct`, remove `/` and `_`
    - Remove all non-alphanumeric characters
  - Build a lookup dict from normalized column name → actual column name in the DataFrame.

- Canonicalize feature columns:
  - For each `canonical` feature in `BASE_FEATURES`:
    - Look up all aliases from `ALIASES[canonical]` plus the canonical name itself.
    - Normalize each alias with `norm(...)` and see if it matches any normalized DataFrame column.
    - When a match is found:
      - If the actual column name differs from `canonical`, add it to `rename_map` to be renamed.
  - Apply `rename_map` to `df` so all matched alias columns are renamed to their canonical feature names.
  - Print out any renamings performed for inspection.

- Add derived **SpeedxBMI** metric:
  - If both `Speed` and `BMI` columns exist in the DataFrame:
    - Compute `SpeedxBMI = Speed * BMI`
    - Append `SpeedxBMI` as a new column.
  - Otherwise, print a message indicating the column(s) are missing and skip the feature.

- Save master feature table:
  - Write the updated DataFrame (with canonicalized feature names and `SpeedxBMI` if available) to:  
    `master_list.csv`
  - Print the first ~40 column names as a quick sanity check.


In [1]:
import re
import sys
import pandas as pd
from pathlib import Path


# ========== CONFIG ==========
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
IN_CSV  = ROOT / "data" / "processed" / "master_creation" / "players_dom_birthday_breakout_elusive_best_final_avg_conf_speed_bmi.csv"
OUT_CSV = ROOT / "data" / "processed" / "master_list.csv"

print("Loading:", IN_CSV)
df = pd.read_csv(IN_CSV)

# ----------------------------- Canonical Features -----------------------------

from src.utils import BASE_FEATURES, INTERACTIONS, ALIASES

# ----------------------------- Name Normalization -----------------------------
def norm(s: str) -> str:
    """Normalize string for fuzzy column matching."""
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = s.replace("%", "pct")
    s = s.replace("/", "")
    s = s.replace("_", "")
    s = s.replace(" ", "")
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# Build lookup table
col_norm_to_actual = {}
for c in df.columns:
    key = norm(c)
    if key not in col_norm_to_actual:
        col_norm_to_actual[key] = c

rename_map = {}

# Match aliases → canonical names
for canonical in BASE_FEATURES:
    alias_list = ALIASES.get(canonical, []) + [canonical]
    for a in alias_list:
        k = norm(a)
        if k in col_norm_to_actual:
            actual = col_norm_to_actual[k]
            if actual != canonical:
                rename_map[actual] = canonical
            break

# Apply renames
if rename_map:
    print("Renaming columns:")
    for old, new in rename_map.items():
        print(f"  {old} -> {new}")
    df = df.rename(columns=rename_map)
else:
    print("No feature columns renamed.")

# ----------------------------- Add SpeedxBMI -----------------------------
if "Speed" in df.columns and "BMI" in df.columns:
    df["SpeedxBMI"] = df["Speed"] * df["BMI"]
    print("Added column: SpeedxBMI = Speed * BMI")
else:
    print("Skipping SpeedxBMI (Speed or BMI missing). Columns present:", df.columns.tolist())

# ----------------------------- SAVE --------------------------------------
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_CSV, index=False)

print("Saved to:", OUT_CSV)
print("Sample columns:")
print(df.columns.tolist()[:40])


Loading: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_creation/players_dom_birthday_breakout_elusive_best_final_avg_conf_speed_bmi.csv
Renaming columns:
  DOM -> DOM+
  RDOM -> RDOM+
  PDOM -> PDOM+
  speed -> Speed
  Break % -> Break%
  draft_round -> Draft Capital
  breakout_age -> Breakout Age
  Drop% -> Drop Rate
Added column: SpeedxBMI = Speed * BMI
Saved to: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_list.csv
Sample columns:
['player_name', 'pos', 'birth_date', 'team', 'conf', 'draft_year', 'Draft Capital', 'breakout_year', 'Breakout Age', '10+', 'ADJ%', 'BTT%', 'Break%', 'CC%', 'CTPRR', 'Comp%', 'DAA', 'DOM+', 'DOM+', 'DRP%', 'Drop Rate', 'EPA/P', 'FUM', 'MTF', 'MTF/A', 'PDOM+', 'PDOM+', 'RDOM+', 'RDOM+', 'REC%', 'TWP%', 'UCTPRR', 'Y/REC', 'Y/RR', 'YAC/R', 'YCO/A', 'YPA', 'YPC', 'YPR', 'aDOT']
